In [1]:
from random import shuffle
from IPython.display import clear_output

In [64]:
def create_deck():
    Deck = [2,3,4,5,6,7,8,9,10,'J','Q','K','A'] * 4
    shuffle(Deck)
    return Deck

In [68]:
Deck = create_deck()
print(Deck)
print(len(Deck))
card = Deck.pop(0)
print(card)
print(Deck.pop(0))
print(len(Deck))

[7, 9, 4, 7, 6, 8, 'A', 5, 3, 'Q', 6, 4, 4, 9, 3, 2, 6, 5, 'A', 'K', 10, 8, 'A', 2, 10, 8, 'K', 'K', 'Q', 'J', 2, 8, 'A', 3, 10, 9, 'J', 9, 6, 10, 'J', 5, 'J', 3, 'Q', 'K', 'Q', 7, 5, 2, 4, 7]
52
7
9
50


In [4]:
# Function that pattern matches the string cards to values, and returns integers in the card value range.

def card_match(card):
    match card:
        case n if (type(n) == int) and n < 12:
            return n
        case 'A':
            return [1,11]
        case 'K' | 'J' | 'Q':
            return 10
        case _:
            print('unknown card given. Launch code activated. Abandon all hope')

In [69]:
# A for fun card summation that handles 'A'.
# Fun to implement recursively

def card_sum(hand, sums = 0):
    if hand == []:
        return sums

    card = card_match(hand[0])

# In the case of an 'A' we have two possibilities, so we run the function
# first with high value, and if it goes beyond 21, we use it's low value instead.

    if type(card) == list:
        simh = card_sum([card[1]] + hand[1::], sums)
        if simh <= 21:
            return simh
        else:
            siml = card_sum([card[0]] + hand[1::], sums)
            return siml
# recursively calls itself with the tail adding the first card in the list
# to the sum.   
    return card_sum(hand[1::], sums + card)



In [73]:
hand = [5, 'A', 'A', 'Q', 'A', 'A']
simphand = [5,4,'J']
lowvalhand = [2,2,2,2,3,3,3,3,'A']
smallhand = ['A','K']
acehand = ['A', 'A', 'A', 'A']
values = ([card_match(i) for i in hand])


print('got:', card_sum(hand), 'expected:', 19)
print('got:', card_sum(simphand),'expected:', 19)
print('got:', card_sum(lowvalhand),'expected:', 21)
print('got:', card_sum(smallhand),'expected:', 21)
print('got:', card_sum(acehand),'expected:', 14)

got: 19 expected: 19
got: 19 expected: 19
got: 21 expected: 21
got: 21 expected: 21
got: 14 expected: 14


In [74]:
# A simple dealer that will always stand at soft 17 or higher.
def dealer(hand, Deck, sums = 0):
    if sums == 0:
        hand_sum = card_sum(hand)
    else:
        hand_sum = sums
    print(hand, hand_sum)
    if hand_sum  < 17:
        card = Deck.pop(0)
        new_sums = card_sum([card], hand_sum)
        new_hand = hand + [card]
        return dealer(new_hand, Deck, new_sums)
    elif hand_sum > 21:
        return ['bust', hand, hand_sum]
    else:
        return ['stand', hand, hand_sum]


In [75]:
dhand = [5,7]
dealer(dhand, Deck)

[5, 7] 12
[5, 7, 4] 16
[5, 7, 4, 7] 23


['bust', [5, 7, 4, 7], 23]

In [8]:
# Game state
# draw cards two for player, one hidden and one shown for dealer
# player can choose an option, with extra possibilities first turn
# player chooses actions until they either hit 21, stand or bust
# Dealer turn to draw until equal to or above 17, but below 22 or hitting over 21
# player and dealer compares cards and determines victor

In [91]:
# Function for checking players hand cases, and for player actions: hit, stand, split, and close game.
def player_turn(hand, sums, Deck, legal_split = True):
    while True:
        match sums:
            case n if n == 21:
                if len(hand) == 2:
                    print('Your hand is: ', hand, sums)
                    print('blackjack')
                    state = 'blackjack'
                    break
                else:
                    print('Your hand is: ', hand, sums)
                    state = 'stand'
                    break
            case n if n > 21:
                sums = card_sum(hand)
                if sums > 21:
                    print('Your hand is: ', hand, sums)
                    print('bust')
                    state = 'bust'
                    break
        
        print('Your hand is: ', hand, sums)

        if len(hand) == 2 and card_sum([hand[0]]) == card_sum([hand[1]]) and legal_split == True:
            choice = input('Would you like to hit, stand, split or close? ')
        else:
            choice = input('Would you like to hit, stand or close? ')

        match choice:
            case 'hit':
                hand = hand + [Deck.pop(0)]
                sums = card_sum([hand[-1]], sums)
            case 'stand':
                state = 'stand'
                break
            case 'split':
                if len(hand) == 2 and card_sum([hand[0]]) == card_sum([hand[1]]):
                    if legal_split == True:
                        hand1 = [hand[0], Deck.pop(0)]
                        hand2 = [hand[1], Deck.pop(0)]
                    
                        print('First hand')
                        hand1, sum1, state1, Deck = player_turn(hand1, card_sum(hand1), Deck, False)
                    
                        print('Second hand')
                        hand2, sum2, state2, Deck = player_turn(hand2, card_sum(hand2), Deck, False)
                    
                        return [hand1,hand2], [sum1, sum2], [state1, state2], Deck
                    else:
                        print('You are only allowed to split once')
                else:
                    print('You can\'t split as either you don\'t have exactly 2 cards, or their values are not identical')
            case 'close':
                state = 'break'
                break
    return hand, sums, state, Deck

def result(player_hand, player_sum, state, dealer_hand, dealer_sum, dealer_fate):
    if state == 'bust':
        print('You busted on', player_sum,', You lose')
    elif state == 'blackjack':
        if len(dealer_hand) == 2 and dealer_sum == 21:
            print('Draw')
        else:
            print('You Win')
    elif state == 'stand':
        if dealer_fate == 'bust':
            print('Dealer busted, you win')
        elif dealer_fate == 'stand':
            if len(dealer_hand) == 2 and dealer_sum == 21:
                print('Dealer blackjack, you lose')
            elif dealer_sum < player_sum:
                print('dealer has:', dealer_sum,',you have:', player_sum)
                print('You win')
            elif dealer_sum > player_sum:
                print('dealer has:', dealer_sum,',you have:', player_sum)
                print('You lose')
            else:
                print('dealer has:', dealer_sum,',you have:', player_sum)
                print('Draw')


In [92]:
# The game function that actually runs the game. Allows for starting hands for player and dealer, and allows a starting deck
# for testing scenarios.
def game(player_hand = [], dealer_hand = [], Deck = create_deck()):
    

    if dealer_hand == []:
        dealer_hand = [Deck.pop(0)]
    dealer_sum = card_sum(dealer_hand)
    
    if player_hand == []:
        player_hand = [Deck.pop(0),Deck.pop(0)]
    player_sum = card_sum(player_hand)

    state = None
    play = True
    gamerun = True

# Gameplay loop consists of running the players turn, then the dealer, comparing their hands, and last asking if player
# would like to play another round.

    while play:

        
        if gamerun == True:
            print('Dealer hand is: ', dealer_hand, dealer_sum)
            
            player_hand, player_sum, state, Deck = player_turn(player_hand, player_sum, Deck)
        
            if state == 'break':
                break

            dealer_fate, dealer_hand, dealer_sum = dealer(dealer_hand, Deck, dealer_sum)
            
            if type(player_hand[0]) == list:
                print('Result for first hand is: ')
                result(player_hand[0], player_sum[0], state[0], dealer_hand, dealer_sum, dealer_fate)
                
                print('Result for second hand is: ')
                result(player_hand[1], player_sum[1], state[1], dealer_hand, dealer_sum, dealer_fate)
            else:
                result(player_hand, player_sum, state, dealer_hand, dealer_sum, dealer_fate)

        gamerun = False
        ans = input('Would you like to play another round? ')
        match ans:
            case 'yes':
                clear_output()
                Deck = create_deck()

                player_hand = []
                dealer_hand = []

                dealer_hand = [Deck.pop(0)]
                dealer_sum = card_sum(dealer_hand)
                player_hand = [Deck.pop(0),Deck.pop(0)]
                player_sum = card_sum(player_hand)
                state = None
                play = True
                gamerun = True
            case 'no':
                play = False


In [93]:
game()

Dealer hand is:  [5] 5
Your hand is:  ['J', 5] 15
Would you like to hit, stand or close? hit
Your hand is:  ['J', 5, 2] 17
Would you like to hit, stand or close? stand
[5] 5
[5, 2] 7
[5, 2, 4] 11
[5, 2, 4, 6] 17
dealer has: 17 ,you have: 17
Draw
Would you like to play another round? no
